# 目标：使用LangChain框架搭建一个简单的数学计算Agent

首先，需要安装相关的python包

In [4]:
!pip install langchain docmesh_agent

# 下面，我们演示一下关键组件的定义

In [5]:
from typing import Type, Optional, List, Callable, Any

from docmesh_agent.tools.base import BaseAgentTool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import tool
from langchain_core.tools import BaseTool, BaseToolkit

# 定义Agent方法一：

In [6]:
# 定义求和Agent的两个输入参数
class AddInput(BaseModel):
    a:int = Field(description="first number")
    b: int = Field(description="second number")

# 定义求和的Agent
class AddTool(BaseTool):
    name = "add"
    description = "Adds two numbers together"
    args_schema: Optional[Type[BaseModel]] = AddInput
    return_direct: bool = True

    def _run(self, a: int, b: int,)->int:
      return a + b

In [7]:
# 实例化求和Agent，并输出对应的参数
add_tool = AddTool()

print(add_tool.name)
print(add_tool.description)
print(add_tool.args)

add
Adds two numbers together
{'a': {'title': 'A', 'description': 'first\xa0number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second\xa0number', 'type': 'integer'}}


# 定义Agent方法二：

还有一种更为简单的方式来定义tool，直接用装饰器

In [8]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together"""
    return a + b

print(add.name)
print(add.description)
print(add.args)

add
Adds two numbers together
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


这里直接会把a和b参数进行包装，且"""之间的即为 description

# math agent示例
实现一个可以做一些数学题的agent

这里只实现两个tool，一个是+一个是✖️

In [9]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together"""
    return a + b


@tool
def multipy(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

定义好tool之后，可以将tool按照toolkit进行管理

In [10]:
class MathToolkit(BaseToolkit):
    def get_tools(self) -> list[BaseAgentTool]:
        return [
            add,
            multipy
        ]

之后完成后面几步，就可以和agent进行沟通了，加上前面的步骤，主要需要实现：

定义tool

用toolkit进行分类管理

实例化llm

创建prompt

利用tool + llm + prompt创建agent

利用agent + tool创建agent executor

In [11]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [12]:
from typing import Type, Optional, List, Callable, Any

from docmesh_agent.tools.base import BaseAgentTool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain_core.tools import BaseToolkit

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together"""
    return a + b


@tool
def multipy(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

class MathToolkit(BaseToolkit):
    def get_tools(self) -> list[BaseAgentTool]:
        return [
            add,
            multipy
        ]


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-2024-05-13") # 这里的llm可以替换为国产大模型

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
          You are a mathematical assistant. Use your tools to answer questions.
          If you do not have a tool to answer the question, say so.

          Return only the answers. e.g
          Human: What is 1 + 1?
          You: 2
          """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

tools = [
    *MathToolkit().get_tools()
]
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "what is (1 + 3) * 6"})

print(result)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 3}`


4
Invoking: `multipy` with `{'a': 4, 'b': 6}`


2424

> Finished chain.
{'input': 'what\xa0is\xa0(1\xa0+\xa03)\xa0*\xa06', 'output': '24'}
